# Tutorial: for search EP4/EP1 workflow data via digitaltwins-on-fhir

## Introduction
The digitaltwins-on-fhir is a package for transfering SPARC SDS dataset to FHIR resource and enable user to use search functions, like search measurements / results accorss multiple datasets.

## Definitions
- FHIR - Fast Healthcare Interoperability Resources
- FHIR Server - a server implemented by FHIR standard, allow users to create / update / delete / search data based on FHIR standard.
- Primary Measurements - the SPARC SDS dataset primary folder and its clinical description annotation (EHR).
- Workflow - a cwl workflow file.
- Workflow tool - a cwl worklfow tool file
- Workflow tool process - a data generated by executing a workflow tool

## Learning outcomes

- Learn how to find primary measurements of a patient across multiple datasets
- Learn how to find MRI images of all relevant datasets of a patient and their pac endpoints.
- Learn how to find a interested result generated by which workflow/workflow tool and relevant to which dataset.
- Learn how to find a wolkflow's all relevant workflow tool processes and workflow tools.
- Learn how to find a specific workflow tool process and its inputs/outputs





Install package


*   pip install digitaltwins-on-fhir

In [ ]:
pip install digitaltwins-on-fhir

In [ ]:
from digitaltwins_on_fhir import Adapter
from pprint import pprint
adapter = Adapter("http://130.216.217.173:8080/fhir")
client = adapter.async_client

## Example 1 Find all primary measurements of a patient across different datasets

Step 1, List all patients and the datasets they are involved in

In [ ]:

patients = await client.resources("Patient").search().fetch_all()
target_patient = None
for p in patients:
  research_subjects = await client.resources("ResearchSubject").search(patient=p.to_reference()).fetch_all()
  dataset = []
  if len(research_subjects) > 1:
    target_patient = p
  for r in research_subjects:
    dataset.append(await r.get("study").to_resource())
  p_uuid = p.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '')
  print(f"The patient: {p_uuid} involves in dataset {dataset}")



 Now we find the patient: 54f2abff-6594-11ef-917d-484d7e9beb16 involved in two dataset

 Step 2, Let's find all primary measurements of this patient and list the measurement comes from which dataset.

In [ ]:
compositions = await client.resources("Composition").search(title="primary measurements",
                                                            author=target_patient.to_reference()).fetch_all()
for c in compositions:
  measurements = c.get_by_path(["section", 0, "entry"])
  dataset = await c.get('subject').to_resource()
  duuid = dataset.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '')
  for m in measurements:
    mr = await m.to_resource()
    code = mr.get_by_path(["code", "coding", 0, "code"])
    print(f"The measurement resource is {mr.get('resourceType')}, and code is {code}, it comes from dataset {dataset}: {duuid}")

We also can find all patient involved datasets' MRI images.

Step 3, List all MRI images (**cross multiple datasets**) of the patient and list the image's pac's endpoint

In [ ]:
images = await client.resources("ImagingStudy").search(subject=target_patient.to_reference()).fetch_all()

for image in images:
  composition = await client.resources("Composition").search(entry=image.to_reference()).first()
  dataset = await composition.get('subject').to_resource()
  duuid = dataset.get_by_path([
      'identifier',
      {'system':'https://www.auckland.ac.nz/en/abi.html'},
      'value'
  ], '')
  print(f"The MRI study {image} comes from the dataset {dataset}: {duuid}")
  image_study_endpoint = await image.get("endpoint")[0].to_resource()
  print(f"The image {image} study pac's address is: {image_study_endpoint.get('address')}")
  for sample in image.get("series"):
    sample_endpoint = await sample.get("endpoint")[0].to_resource()
    print(f"The image {image} sample pac's address is: {sample_endpoint.get('address')}")
  print("-----------------------------------------------------------------------------------\n")

## Example 2 Find the result observation is generated by which workflow and related to which dataset

1 I'm interested in a result observation, e.g, distance to nipple

And the uuid is: b6b7b363-65ae-11ef-917d-484d7e9beb16-workflow-tool-6-process-002-Observation-0



*   Let's find which workflow / wolkflow tool generate this observation
*   Let's find the observation related to which dataset


In [ ]:
ob = await client.resources("Observation").search(
            identifier="b6b7b363-65ae-11ef-917d-484d7e9beb16-workflow-tool-6-process-002-Observation-0").first()
print("Step 0: We are interested in this observation (Tumour distance to nipple): ", ob)

Let's find the observation is generated by which workflow

In [ ]:
# step 1, Find this observation is assigned in which workflow result composition
composition = await client.resources("Composition").search(title="workflow tool results",
                                                            entry=ob.to_reference().reference).first()
print("Step 1: the interested observation is assigned in this composition: ", composition)
# step 2, Let's find which workflow tool process generates the composition
workflow_tool_process = await composition.get("subject").to_resource()
print("Step 2: the composition is generated by this workflow_tool_process: ", workflow_tool_process)
# step 3, Now we can find the observation is related to which workflow
workflow = await workflow_tool_process.get("for").to_resource()
print("Step 3: Now we find the interested observation is related to this workflow: ", workflow)
# step 3.1, Because we know the interested observation, so we can directly find which workflow tool generates it
workflow_tool = await ob.get("focus")[0].to_resource()
print("Step 3.1: We also can find the interested observation is related to this workflow_tool: ", workflow_tool)

Let's also find the interested observation is related which dataset

In [ ]:
# step 4, let's find which research subject is related to this observation
research_subject = await workflow_tool_process.get("basedOn")[0].to_resource()
print("Step 4, the interested observation is related to this research_subject: ", research_subject)
# step 4.1, we also can find this observation is generated by which patient
patient = await composition.get("author")[0].to_resource()
print("Step 4.1: the observation comes from this patient: ", patient)
# step 5, now we can know the observation is related to which dataset
dataset = await research_subject.get("study").to_resource()
print("Step 5, Now we find the interested observation is related to this dataset: ", dataset)

## Example 3 Find all workflow tools and workflow tool processes related to the workflow

2 If we know the workflow uuid, let's find the workflow

Let's find all workflows, and then choose an uuid for remain tasks

In [ ]:
# Get all workflows
workflows =  await client.resources("PlanDefinition").search().fetch_all()
print("All workflow resources:", workflows)
print("Workflows' uuids: ", [w.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '') for w in workflows ])


Also we can find the breast workflow by search `name` or `title`

In [ ]:
breast_workflow = await client.resources("PlanDefinition").search(name="breast workflow").fetch_all()
print("The breast workflow:", breast_workflow)
print("uuid: ", breast_workflow[0].get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ]))

2.1 let's find all related workflow tools of breast workflow

In [ ]:
workflow = await client.resources("PlanDefinition").search(identifier="e3b3eaa0-65ae-11ef-917d-484d7e9beb16").first()
actions = workflow.get("action")
for a in actions:
  if a.get("definitionCanonical") is None:
      continue
  resource_type, _id = a.get("definitionCanonical").split("/")
  workflow_tool = await client.reference(resource_type, _id).to_resource()
  print(f"workflow tool name: {workflow_tool.get('title')}", workflow_tool)
  # TODO 2.1.1 Find model and software uuid from a specific workflow tool
  print("The software and model uuids:")
  pprint(workflow_tool.get("participant"))
  print("---------------------------------")

2.2 let's find all related workflow tool process for that workflow

In [ ]:
workflow_tool_processes = await client.resources("Task").search(
            subject=workflow.to_reference().reference).fetch_all()
pprint(workflow_tool_processes)

## Example 4 Find a specific workflow tool process for a patient in a dataset
This requires:

*   patient uuid: 54f2abff-6594-11ef-917d-484d7e9beb16
*   workflow uuid: e3b3eaa0-65ae-11ef-917d-484d7e9beb16
*   workflow tool uuid: b6b7b363-65ae-11ef-917d-484d7e9beb16
*   and dataset uuid: 93d49efa-5f4e-11ef-917d-484d7e9beb16



In [ ]:
# Step 1: find the patient
patient = await client.resources("Patient").search(identifier="54f2abff-6594-11ef-917d-484d7e9beb16").first()
# Step 2: find the dataset
dataset = await client.resources("ResearchStudy").search(identifier="93d49efa-5f4e-11ef-917d-484d7e9beb16").first()
# Step 3: find the workflow tool
workflow_tool = await client.resources("ActivityDefinition").search(
    identifier="b6b7b363-65ae-11ef-917d-484d7e9beb16").first()
# Step 4: find the research subject
research_subject = await client.resources("ResearchSubject").search(patient=patient.to_reference().reference,
                                                                    study=dataset.to_reference().reference).first()
print("research_subject: ", research_subject)
workflow_tool_process = await client.resources("Task").search(subject=workflow.to_reference(),
                                                              focus=workflow_tool.to_reference(),
                                                              based_on=research_subject.to_reference(),
                                                              owner=patient.to_reference()).first()
print("Now the workflow_tool_process is: ", workflow_tool_process)

Now we get the breast workflow's step 7 workflow tool's workflow tool process which belongs to patient 1 in dataset 1

3.1 Get the input of the workflow tool process

In [ ]:
inputs = workflow_tool_process.get("input")
for i in inputs:
    composition = await client.resources("Composition").search(title="primary measurements",
                                                                entry=i.get("valueReference")).first()
    print("primary measurement's composition: ", composition)
    dataset = await composition.get("subject").to_resource()
    print("dataset: ", dataset)

3.2 We also can list all outputs of the workflow tool process

In [ ]:
outputs = workflow_tool_process.get("output")
pprint(outputs)

3.3 We also can list all output resources

In [ ]:
for o in outputs:
  output_resource = await o.get("valueReference").to_resource()
  print(output_resource)